## TIRCP DRISI
* DRISI emailed a request asking for TIRCP outcomes for cycles 3-5.  
* [Cycles 1-6](https://calsta.ca.gov/subject-areas/transit-intercity-rail-capital-prog)
* Cycle 1: 2015
* Cycle 2: 2016
* Cycle 3: 2018
* Cycle 4: 2020
* Cycle 5: 2022
* Cycle 6: 2023

In [116]:
import A1_data_prep
import A2_tableau
import A6_zev
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

In [117]:
pd.options.display.max_columns = 125
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [118]:
# GCS File Path:
GCS_FILE_PATH = A1_data_prep.GCS_FILE_PATH

In [151]:
def read_in_files(file_name: str, columns_to_drop: list):
    df = to_snakecase(pd.read_excel(f"{A1_data_prep.GCS_FILE_PATH}{file_name}"))

    df = df.drop(columns=columns_to_drop)

    df = df.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))

    return df

### Manipulate TIRCP Project Sheet

In [168]:
def clean_tircp():
    df_tircp = to_snakecase(A2_tableau.tableau_dashboard())

    # Drop irrelevant years.
    df_tircp2 = df_tircp.loc[df_tircp["award_year"] >= 2018].reset_index(drop=True)

    # Sort df by award year and number
    df_tircp2 = df_tircp2.sort_values(["award_year", "#"])

    # Create a detailed title column to avoid duplicates by combining  year,title, and recipient into one line.
    df_tircp2["award_year"] = df_tircp2["award_year"].astype("object")

    # https://stackoverflow.com/questions/39291499/how-to-concatenate-multiple-column-values-into-a-single-column-in-pandas-datafra'
    detailed_title_cols = ["award_year", "title", "grant_recipient"]
    df_tircp2["detailed_title_col"] = df_tircp2[detailed_title_cols].apply(
        lambda row: "-".join(row.values.astype(str)), axis=1
    )

    # Subset df to columns of interest
    df_tircp2 = df_tircp2[
        [
            "award_year",
            "#",
            "ppno",
            "tircp",
            "title",
            "detailed_title_col",
            "grant_recipient",
            "district",
            "county",
            "description",
            "total__cost",
            "estimated_tircp_ghg_reductions",
            "increased_ridership",
            "service_integration",
            "improve_safety",
        ]
    ]

    df_tircp2["project_number"] = (
        df_tircp2["award_year"].astype(str) + "-" + "0" + df_tircp2["#"].astype(str)
    )

    return df_tircp2

In [171]:
tircp = clean_tircp()

In [172]:
tircp.head(1)

,award_year,#,ppno,tircp,title,detailed_title_col,grant_recipient,district,county,description,total__cost,estimated_tircp_ghg_reductions,increased_ridership,service_integration,improve_safety,project_number
0,2018,1,2320B,14000000.00,Purchase Zero Emission High Capacity Buses to Support Transbay Tomorrow and Clean Corridors Plan,2018-Purchase Zero Emission High Capacity Buses to Support Transbay Tomorrow and Clean Corridors Plan-Alameda Contra Costa Transit District,Alameda Contra Costa Transit District,District 4: Bay Area / Oakland,Alameda,Zero-Emission Buses for Service Expansion,67645000.00,"14,000 MTCO2e",Medium,Medium,Medium,2018-01


### TIRCP Battery and Hydrogen Fuel Cell Bus list 10-10-2022 (1).xlsx

In [161]:
battery = to_snakecase(
    pd.read_excel(
        f"{GCS_FILE_PATH}DRISI/TIRCP Battery and Hydrogen Fuel Cell Bus list 10-10-2022 (1).xlsx"
    )
)

In [164]:
battery_drop_cols = [
    "local_agency_contact",
    "local_agency_email",
    "local_agency_phone_number",
    "awarded_allocated",
    "procured_contracted",
    "components",
]

In [165]:
battery = battery.drop(columns=battery_drop_cols)

In [178]:
battery = battery.fillna(outcomes.dtypes.replace({"float64": 0.0, "object": "None"}))

In [179]:
battery["project_number"] = (
    battery["award_year"].astype(str) + "-" + "0" + battery["project_#"].astype(str)
)

In [180]:
battery.sample()

,award_year,project_#,grant_recipient,implementing_agency,procured_contracted,in_operation,#_hydrogen_fuel_cell_buses,#_battery_electric_buses,components,project_number
26,2020,15,Torrance Transit Department,Torrance Transit Department,2021-11-16,NaN,NaN,7.00,Procurement of Zero-Emission Electric Buses,2020-015


In [177]:
tircp.merge(battery, how="outer", on=["project_number"], indicator=True)[
    ["_merge"]
].value_counts()

_merge    
left_only     49
both          22
right_only     6
dtype: int64

### AwardedProjectsDetail.xlsx

In [129]:
drisi = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}DRISI/AwardedProjectsDetail.xlsx"))

In [130]:
drisi_drop_cols = [
    "agency_code",
    "agency_short_name",
    "sub_program_description",
    "agency_name",
    "program_short_name",
    "program_name",
    "program_description",
    "sub_program_short_name",
    "sub_program_name",
    "project_type",
    "agency",
    "program",
    "date_imported",
]

In [131]:
drisi = drisi.drop(columns=drisi_drop_cols)

In [132]:
drisi = drisi.fillna(drisi.dtypes.replace({"float64": 0.0, "object": "None"}))

In [133]:
# drisi.info()

In [134]:
# drisi.sample()

### ImplementedProjectsDetail.xlsx

In [135]:
implemented = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}DRISI/ImplementedProjectsDetail.xlsx")
)

In [136]:
# list(implemented.columns)

In [137]:
# implemented.sample()

In [138]:
implemented_drop_cols = [
    "programuniqueidentifier",
    "record_type",
    "reporting_cycle_name",
    "agency_short_name",
    "agency_name",
    "date_operational",
    "program_name",
    "program_description",
    "project_completion_date",
    "date_imported",
    "sub_program_name",
    "date_selected_for_award",
    "project_name",
    "project_type",
    "fiscal_year_funding_project",
    "census_tract",
    "address",
    "lat_long",
    "total_program_ggrffunding",
    "voucher_name",
    "voucher_description",
]

In [139]:
implemented = implemented.drop(columns=implemented_drop_cols)

In [140]:
implemented = implemented.fillna(
    implemented.dtypes.replace({"float64": 0.0, "object": "None"})
)

In [114]:
# implemented.info(verbose=True, show_counts=True)

### TIRCP_AllProjects_12212022 (002).xlsx
* Doesn't have anything interesting.

In [141]:
all_projects = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}DRISI/TIRCP_AllProjects_12212022 (002).xlsx")
)

In [16]:
# Read in sheet with Assembly info.
gis = to_snakecase(
    pd.read_excel(
        f"{GCS_FILE_PATH}TIRCP_GIS_Template_Requirements 6-1-2022.xlsx",
        sheet_name="Projects Table",
    )
)

In [17]:
# Clean some column names
gis = gis.rename(
    columns={
        "ppno_": "ppno",
    }
)

In [18]:
# Clean PPNO
gis = A1_data_prep.ppno_slice(gis)

In [19]:
# Subset for only cols of interest
gis2 = gis[
    [
        "project_number",
        "ppno",
        "projecttitle",
        "projectstatus",
    ]
]

In [20]:
gis2.ppno.nunique()

45

In [21]:
# There are mulitple entries for each ppno.
gis2.ppno.value_counts().head()

CP033    60
CP035    21
CP042    18
CP032    14
CP031    11
Name: ppno, dtype: int64

In [22]:
# Inglewood Transit Center coded as CP063, should be CP062 per TIRCP Tracking sheet.
gis2.loc[
    (gis2["projecttitle"] == "Inglewood Transit Center (2020:04)"), "ppno"
] = "CP062"

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
# North State Intercity Bus System coded as CP063 per TIRCP Tracking sheet.
gis2.loc[
    (
        gis2["projecttitle"]
        == "North State Intercity Bus System-Lake County Interregional Transit Center (2020:05)"
    ),
    "ppno",
] = "CP063"

In [24]:
# Clean project_number, only keep year
gis2["project_number"] = gis2["project_number"].str.split(":").str[0]

/tmp/ipykernel_1188/1194175043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
gis2["project_number"] = gis2["project_number"].fillna(0).astype("int64")

/tmp/ipykernel_1188/511736485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
project_status_gis = A6_zev.summarize_rows(gis2, "ppno", "projectstatus")

In [27]:
# Check that each row matches the number of unique ppno
len(project_status_gis) == gis2.ppno.nunique()

True

In [28]:
# Drop old project status from original dataframe
gis2 = gis2.drop(columns=["projectstatus"])

In [29]:
# Merge with original gis, so there is only one row for each PPNO
final_gis_template = (
    pd.merge(project_status_gis, gis2, how="left", on=["ppno"])
    .drop_duplicates("ppno")
    .reset_index(drop=True)
)

In [30]:
# Checks
len(final_gis_template), final_gis_template.ppno.nunique(), gis.ppno.nunique()

(45, 45, 45)

##### Merge GIS information 

In [31]:
# Merge with df_tircp2
merge1 = pd.merge(
    df_tircp2,
    final_gis_template,
    how="left",
    left_on=["ppno", "award_year"],
    right_on=["ppno", "project_number"],
    indicator=True,
)

In [32]:
merge1._merge.value_counts()

both          43
left_only     25
right_only     0
Name: _merge, dtype: int64

In [33]:
# Double Check that titles & years correspond with one another
merge1[["title", "projecttitle", "projectstatus"]].sample(2)

,title,projecttitle,projectstatus
52,The Regional Connectivity Improvement Bus Program,NaN,NaN
31,Inglewood Transit Connector Project,Inglewood Transit Center (2020:04),"Construction, PA&ED"


In [34]:
# Drop merge & GIS Template projects
merge1 = merge1.drop(columns=["_merge", "projecttitle"])

In [35]:
gis_tracking_sheet = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}{A1_data_prep.FILE_NAME }", sheet_name="GIS Info")
)

In [36]:
# Drop certain cols
gis_tracking_sheet = gis_tracking_sheet[
    [
        "award_year",
        "project_title",
        "caltransdistrict",
        "assembly\ndistricts",
        "senate\ndistricts",
        "city_code",
        "county_code",
        "_implementing_agency__id_",
    ]
]

In [37]:
# Only keep the years wanted
gis_tracking_sheet2 = gis_tracking_sheet.loc[
    gis_tracking_sheet["award_year"] >= 2018
].reset_index(drop=True)

In [38]:
# Merge with df_tircp2
merge2 = pd.merge(
    merge1,
    gis_tracking_sheet2,
    how="left",
    left_on=["award_year", "title"],
    right_on=["award_year", "project_title"],
    indicator=True,
)

In [39]:
# Merge
merge2._merge.value_counts()

both          67
left_only      1
right_only     0
Name: _merge, dtype: int64

In [40]:
merge2.shape, df_tircp2.shape

((68, 26), (68, 16))

In [41]:
merge2 = merge2.drop(columns=["project_number", "project_title", "_merge"])

In [42]:
# Copy merge 2
projects = merge2.copy()

In [43]:
# Fill in empty values with NA
projects = projects.fillna(
    projects.dtypes.replace({"float64": 0.0, "object": "None", "int64": 0})
)

In [44]:
# Format moentary cols
monetary_cols = ["total__cost", "tircp"]
for i in monetary_cols:
    projects[i] = projects[i].apply(
        lambda x: format_currency(x, currency="USD", locale="en_US")
    )

In [45]:
# Clean up column names
projects = A1_data_prep.clean_up_columns(projects)

In [46]:
projects = projects.rename(
    columns={
        "Number Use": "Project Number",
        "Assembly\nDistricts": "Assembly Districts",
        "Senate\nDistricts": "Senate Districts",
        "Caltransdistrict": "CT Districts",
        "Assembly\nDistricts": "Assembly Districts",
    }
)

In [47]:
# Rearrange columns
right_order = [
    "Award Year",
    "#",
    "Project Number",
    "Ppno",
    "Title",
    "Grant Recipient",
    "Tircp",
    "Total  Cost",
    "Description",
    "District",
    "County",
    "Status",
    "CT Districts",
    "Assembly Districts",
    "Senate Districts",
    "City Code",
    "County Code",
    "Implementing Agency  Id",
]

In [48]:
projects = projects[right_order]

### OutcomeProjectsDetail.xlsx 

In [143]:
outcomes = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}DRISI/OutcomeProjectsDetail.xlsx")
)

In [148]:
outcomes_drop_cols = [
    "unnamed:_0",
    "proj_rec_id",
    "reporting_cycle_name",
    "agency_short_name",
    "agency_name",
    "program_short_name",
    "program_name",
    "program_description",
    "record_type",
    "sub_program_short_name",
    "sub_program_name",
    "sub_program_description",
    "date_imported",
]

In [149]:
outcomes = outcomes.drop(columns=outcomes_drop_cols)

In [147]:
# list(outcomes.columns)

In [150]:
outcomes = outcomes.fillna(outcomes.dtypes.replace({"float64": 0.0, "object": "None"}))

### Outcomes Sheet

In [49]:
# Measure columns
measure_cols = [
    "estimated_tircp_ghg_reductions",
    "cost_per_ghg_ton_reduced",
    "increased_ridership",
    "service_integration",
    "improve_safety",
]

In [50]:
# Turn estimated GHG reductions into a number
merge2["estimated_tircp_ghg_reductions"] = (
    merge2["estimated_tircp_ghg_reductions"]
    .str.replace("MTCO2e", "")
    .str.replace("None", "")
    .str.replace(",", "")
)

In [51]:
merge2["estimated_tircp_ghg_reductions"] = (
    merge2["estimated_tircp_ghg_reductions"]
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
)

In [52]:
# Subset to cols similar to SCCP
outcomes = merge2[
    [
        "award_year",
        "detailed_title_col",
        "estimated_tircp_ghg_reductions",
        "increased_ridership",
        "service_integration",
        "improve_safety",
    ]
].sort_values(
    [
        "award_year",
        "detailed_title_col",
    ]
)

In [53]:
outcomes = A1_data_prep.clean_up_columns(outcomes)

In [54]:
outcomes.head(1)

,Award Year,Detailed Title Col,Estimated Tircp Ghg Reductions,Increased Ridership,Service Integration,Improve Safety
1,2018,2018-#Electrify Anaheim: Changing the Transit Paradigm in Southern California-Anaheim Transportation Network,61000.00,Medium-High,Medium-High,Medium


##### Version 1

In [55]:
# Drop award year
outcomes_transformed = outcomes.drop(columns=["Award Year"]).T

In [56]:
# Make first row to column names
outcomes_transformed.columns = outcomes_transformed.iloc[0]

In [57]:
# Del first row
outcomes_transformed = outcomes_transformed.iloc[1:]

In [58]:
outcomes_transformed.head()

Detailed Title Col,2018-#Electrify Anaheim: Changing the Transit Paradigm in Southern California-Anaheim Transportation Network,2018-Accelerating Rail Modernization and Expansion in the Capital Region-Sacramento Regional Transit District,2018-All Aboard 2018: Transforming SoCal Rail Travel-Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,2018-Blue Line Rail Corridor Transit Enhancements-San Diego Metropolitan Transit System,2018-Building Up: LOSSAN North Improvement Program-Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,2018-Coastal Express/Pacific Surfliner Peak Hour Service Expansion and Integration Project-Santa Barbara County Association Of Governments,2018-Diesel Multiple Unit Vehicle to Zero- or Low-Emission Vehicle Conversion and West Valley Connector Bus Rapid Transit-San Bernardino County Transportation Authority,2018-Dublin/Pleasanton Capacity Improvement and Congestion Reduction Program-Livermore Amador Valley Transit Authority,2018-Electric Blue: Electrification of City of Santa Monica's Big Blue Bus-City Of Santa Monica,2018-Extend rail service to Monterey County-Transportation Agency For Monterey County,2018-From the Desert to the Sea: Antelope Valley Transit Authority and Long Beach Transit Zero Emission Bus Initiative-Antelope Valley Transit Authority,2018-Goleta Train Depot-Santa Barbara County Association Of Governments,2018-Los Angeles City: Leading the Transformation to Zero-Emission Electric Bus Transit Service-City Of Los Angeles,2018-Los Angeles Region Transit System Integration and Modernization Program of Projects-Los Angeles County Metropolitan Transportation Authority,2018-North State Intercity Bus System-Shasta Regional Transportation Agency,2018-Peninsula Corridor Electrification Expansion Project-Peninsula Corridor Joint Powers Board,2018-Purchase Zero Emission High Capacity Buses to Support Transbay Tomorrow and Clean Corridors Plan-Alameda Contra Costa Transit District,2018-Ride Between the Line: Enhancing Access to Transit in San Diego-San Diego Association Of Governments,2018-SMART Larkspur to Windsor Corridor-Sonoma-Marin Area Rail Transit District,2018-SamTrans Express Bus Pilot-San Mateo County Transit District,2018-Solano Regional Transit Improvements-Solano Transportation Authority,2018-Southern California Optimized Rail Expansion (SCORE)-Southern California Regional Rail Authority,2018-Southwest Fresno Community Connector-City Of Fresno,2018-The Northern California Corridor Enhancement Program-Capitol Corridor Joint Powers Authority,2018-The Transbay Corridor Core Capacity Program: Vehicle Acquistion and Communications-Based Train Control System-Bay Area Rapid Transit,2018-Transit Capacity Expansion Program-San Francisco Municipal Transportation Agency,"2018-VTA’s BART Silicon Valley Extension, Phase II-Santa Clara Valley Transportation Authority",2018-Valley Rail-San Joaquin Joint Powers Authority,2020-Building Up Control: LOSSAN Service Enhancement Program-Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,2020-Core Capacity Program-San Francisco Municipal Transportation Agency,2020-Expansion of WETA Ferry Services-San Francisco Bay Area Water Emergency Transportation Authority,"2020-For People, Place and Planet: Connecting Inglewood to Regional Opportunities-Santa Monica Big Blue Bus","2020-Improving Air Quality & Economic Growth with Electric Buses in Merced County, the Gateway to Yosemite-Transit Joint Powers Authority Of Merced County",2020-Inglewood Transit Connector Project-City Of Inglewood,2020-LBT/UCLA Electric Commuter Express-Long Beach Transit,2020-Light Rail Modernization and Expansion of Low-Floor Fleet-Sacramento Regional Transit District,2020-Metrolink Antelope Valley Line Capital and Service Improvements-Los Angeles County Metropolitan Transportation Authority,2020-North State Intercity Bus System-Lake Transit Authority,"2020-Reaching the Most Transit-Vulnerable: AVTA's Zero Emission ""Microtransit"" & Bus Expansion Proposal-Antelope Valle

##### Outputs: Measures except GHG Reductions.

In [59]:
outcomes_melt = pd.melt(
    outcomes,
    id_vars=[
        "Award Year",
        "Detailed Title Col",
    ],
    value_vars=[
        "Increased Ridership",
        "Service Integration",
        "Improve Safety",
    ],
)

In [60]:
outcomes_melt = A1_data_prep.clean_up_columns(outcomes_melt)

In [61]:
year_summary = (
    outcomes_melt.groupby(["Award Year", "Variable", "Value"])
    .agg({"Detailed Title Col": "nunique"})
    .rename(columns={"Detailed Title Col": "Number of Projects in this Value Category"})
)

In [62]:
year_summary

Number of Projects in this Value Category
Award Year Variable            Value                                                 
2018       Improve Safety      High                                                 9
                               Medium                                              12
                               Medium-High                                          7
           Increased Ridership High                                                13
                               Medium                                              10
                               Medium-High                                          5
           Service Integration High                                                 5
                               Medium                                               8
                               Medium-High                                         15
2020       Improve Safety      High                                                 4
                               Medium                                               7
                               Medium-High                                          6
           Increased Ridership High                                                 1
                               Medium                                               4
                               Medium-High                                          9
                               Medium-Low                                           3
           Service Integration High                                                 2
                               Medium                                               5
                               Medium-High                                          8
                               Medium-Low                                           2
2022       Improve Safety      High                                                 8
                               Medium                                               8
                               Medium-High                                          6
                               Medium-Low                                           1
           Increased Ridership High                                                 6
                               Medium                                               7
                               Medium-High                                          8
                               Medium-Low                                           2
           Service Integration High                                                 5
                               Medium                                               8
                               Medium-High                                          7
                               Medium-Low                                           3

In [63]:
GHG_by_year = outcomes.groupby(["Award Year"]).agg(
    {"Estimated Tircp Ghg Reductions": "sum"}
)

In [64]:
GHG_by_year

,Estimated Tircp Ghg Reductions
Award Year,
2018,31944000.00
2020,5016000.00
2022,4332000.00


#### Save

In [65]:
"""
with pd.ExcelWriter(f"{GCS_FILE_PATH}calsta_draft.xlsx") as writer:
    outcomes.to_excel(writer, sheet_name="outcomes_unpivoted", index=True)
    outcomes_transformed.to_excel(writer, sheet_name="outcomes_transformed", index=True)
    projects.to_excel(writer, sheet_name="projects", index=True)
    year_summary.to_excel(writer, sheet_name="year_summary", index=True)
    GHG_by_year.to_excel(writer, sheet_name="GHG_reduction_year", index=True)
    """

'\nwith pd.ExcelWriter(f"{GCS_FILE_PATH}calsta_draft.xlsx") as writer:\n    outcomes.to_excel(writer, sheet_name="outcomes_unpivoted", index=True)\n    outcomes_transformed.to_excel(writer, sheet_name="outcomes_transformed", index=True)\n    projects.to_excel(writer, sheet_name="projects", index=True)\n    year_summary.to_excel(writer, sheet_name="year_summary", index=True)\n    GHG_by_year.to_excel(writer, sheet_name="GHG_reduction_year", index=True)\n    '